# Aula 09 - Pré-processamento: Limpeza e Tratamento de Dados
**Curso:** Programação para Ciência de Dados  
**Aluno:** [Seu Nome Aqui]  
**Data:** [Data Atual]

## Instruções
1. Este notebook demonstra o **pipeline completo de limpeza de dados**
2. Execute as células sequencialmente
3. Complete os exercícios marcados com `# === SEU CÓDIGO AQUI ===`
4. Execute as células de teste para verificar suas respostas
5. **Tempo estimado:** 20-30 minutos

## Objetivos de Aprendizado
- Identificar e classificar dados faltantes (MCAR, MAR, MNAR)
- Aplicar estratégias apropriadas de imputação
- Limpar inconsistências de strings e formatos
- Validar ranges e tipos de dados
- Construir pipeline de limpeza reproduzível

In [ ]:
# === CONFIGURAÇÃO INICIAL ===

# Atualizar pip e limpar cache
!pip install --upgrade pip --quiet
!pip cache purge

# Instalar Otter Grader
!pip install otter-grader --no-cache-dir -q

# Criar diretório para testes
!mkdir -p tests

print("Otter Grader instalado!")
print("Diretório de testes criado!")

In [ ]:
%%writefile tests/p1.py
OK_FORMAT = True

test = {
    "name": "p1",
    "points": 1,
    "suites": [
        {
            "cases": [
                {
                    "code": r"""
>>> # Teste 1: Verificar se o relatório foi criado
>>> import pandas as pd
>>> isinstance(missing_report, pd.DataFrame)
True
""",
                    "hidden": False,
                    "locked": False
                },
                {
                    "code": r"""
>>> # Teste 2: Verificar colunas do relatório
>>> 'Missing_Count' in missing_report.columns and 'Missing_Pct' in missing_report.columns
True
""",
                    "hidden": False,
                    "locked": False
                }
            ],
            "scored": True,
            "setup": "",
            "teardown": "",
            "type": "doctest"
        }
    ]
}

In [ ]:
%%writefile tests/p2.py
OK_FORMAT = True

test = {
    "name": "p2",
    "points": 1,
    "suites": [
        {
            "cases": [
                {
                    "code": r"""
>>> # Teste 1: Verificar se flags foram criadas
>>> 'price_was_missing' in df.columns
True
""",
                    "hidden": False,
                    "locked": False
                },
                {
                    "code": r"""
>>> # Teste 2: Verificar tipo das flags
>>> df['price_was_missing'].dtype in ['int64', 'int32', 'uint8']
True
""",
                    "hidden": False,
                    "locked": False
                }
            ],
            "scored": True,
            "setup": "",
            "teardown": "",
            "type": "doctest"
        }
    ]
}

In [ ]:
%%writefile tests/p3.py
OK_FORMAT = True

test = {
    "name": "p3",
    "points": 1,
    "suites": [
        {
            "cases": [
                {
                    "code": r"""
>>> # Teste 1: Verificar se strings foram limpas
>>> bool(df['condition'].str.contains(' ').sum() == 0)
True
""",
                    "hidden": False,
                    "locked": False
                },
                {
                    "code": r"""
>>> # Teste 2: Verificar se foi padronizado para minúsculo
>>> bool(df['condition'].str.islower().all())
True
""",
                    "hidden": False,
                    "locked": False
                }
            ],
            "scored": True,
            "setup": "",
            "teardown": "",
            "type": "doctest"
        }
    ]
}

In [ ]:
%%writefile tests/p4.py
OK_FORMAT = True

test = {
    "name": "p4",
    "points": 1,
    "suites": [
        {
            "cases": [
                {
                    "code": r"""
>>> # Teste 1: Verificar se age e area são numéricos
>>> import pandas as pd
>>> pd.api.types.is_numeric_dtype(df['age']) and pd.api.types.is_numeric_dtype(df['area'])
True
""",
                    "hidden": False,
                    "locked": False
                },
                {
                    "code": r"""
>>> # Teste 2: Verificar se valores inválidos foram tratados
>>> bool((df['age'] < 0).sum() == 0)
True
""",
                    "hidden": False,
                    "locked": False
                }
            ],
            "scored": True,
            "setup": "",
            "teardown": "",
            "type": "doctest"
        }
    ]
}

In [ ]:
%%writefile tests/p5.py
OK_FORMAT = True

test = {
    "name": "p5",
    "points": 1,
    "suites": [
        {
            "cases": [
                {
                    "code": r"""
>>> # Teste 1: Verificar se não há valores faltantes em price
>>> bool(df_clean['price'].isnull().sum() == 0)
True
""",
                    "hidden": False,
                    "locked": False
                },
                {
                    "code": r"""
>>> # Teste 2: Verificar se o dataset tem menos missing
>>> bool(df_clean.isnull().sum().sum() < df_original.isnull().sum().sum())
True
""",
                    "hidden": False,
                    "locked": False
                }
            ],
            "scored": True,
            "setup": "",
            "teardown": "",
            "type": "doctest"
        }
    ]
}

In [ ]:
# === CARREGAR OTTER GRADER ===
import otter
grader = otter.Notebook()
print("Otter Grader carregado!")

In [ ]:
# === IMPORTAR BIBLIOTECAS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Configurações de plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas com sucesso!")

---
# FASE 1: Auditoria Inicial (Assess)
**Objetivo:** Entender o dataset e identificar problemas iniciais

In [ ]:
# === CARREGAR DADOS ===
try:
    df = pd.read_csv('housing_prices_dirty.csv')
    print("✓ Dataset carregado com sucesso!")
except FileNotFoundError:
    print("⚠ Arquivo não encontrado. Por favor, faça o upload do arquivo 'housing_prices_dirty.csv'")
    print("\nPara fazer upload no Colab, execute:")
    print("from google.colab import files")
    print("uploaded = files.upload()")
    print("df = pd.read_csv(list(uploaded.keys())[0])")

In [ ]:
# === INSPEÇÃO INICIAL ===
print("=" * 60)
print("DIMENSÕES DO DATASET")
print("=" * 60)
print(f"Linhas: {df.shape[0]:,}")
print(f"Colunas: {df.shape[1]}")
print(f"\nMemória usada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n" + "=" * 60)
print("PRIMEIRAS LINHAS")
print("=" * 60)
display(df.head(10))

In [ ]:
# === INFORMAÇÕES DO DATASET ===
print("=" * 60)
print("TIPOS DE DADOS E MISSING VALUES")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("ESTATÍSTICAS DESCRITIVAS")
print("=" * 60)
display(df.describe())

---
# FASE 2: Diagnóstico (Diagnose)
**Objetivo:** Identificar e quantificar todos os problemas

## 📝 Exercício 1: Criar Relatório de Missing Values

Complete a função abaixo para criar um relatório detalhado de valores faltantes:

In [ ]:
# === EXERCÍCIO 1: RELATÓRIO DE MISSING ===

def generate_missing_report(df):
    """
    Gera relatório completo de valores faltantes

    Parameters:
    -----------
    df : DataFrame
        Dataset a ser analisado

    Returns:
    --------
    DataFrame com colunas: Column, Missing_Count, Missing_Pct, Dtype
    """
    # === SEU CÓDIGO AQUI ===
    # Calcule missing_count (soma de valores nulos por coluna)
    missing_count = df.isnull().sum()

    # Calcule missing_pct (percentual de valores nulos)
    missing_pct = (missing_count / len(df)) * 100

    # Crie o DataFrame report com as colunas especificadas
    report = pd.DataFrame({
        'Column': df.columns,
        'Missing_Count': missing_count.values,
        'Missing_Pct': missing_pct.values,
        'Dtype': df.dtypes.values
    })

    # Filtre apenas colunas com missing > 0
    report = report[report['Missing_Count'] > 0]

    # Ordene por percentual decrescente
    report = report.sort_values('Missing_Pct', ascending=False)

    return report

# Gerar e exibir o relatório
missing_report = generate_missing_report(df)
print("\n" + "=" * 60)
print("RELATÓRIO DE VALORES FALTANTES")
print("=" * 60)
display(missing_report)

In [ ]:
# === TESTAR EXERCÍCIO 1 ===
grader.check("p1")

In [ ]:
# === VISUALIZAR PADRÃO DE MISSING ===
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico 1: Percentual de missing por coluna
if len(missing_report) > 0:
    missing_report.plot(x='Column', y='Missing_Pct', kind='barh', ax=axes[0], color='coral')
    axes[0].set_xlabel('Percentual de Missing (%)')
    axes[0].set_ylabel('Coluna')
    axes[0].set_title('Percentual de Missing por Coluna')
    axes[0].grid(axis='x', alpha=0.3)

# Gráfico 2: Heatmap de missing
missing_matrix = df.isnull().astype(int)
sns.heatmap(missing_matrix.T, cmap='RdYlGn_r', cbar=True, ax=axes[1],
            yticklabels=df.columns, xticklabels=False)
axes[1].set_title('Padrão de Missing (vermelho = faltante)')
axes[1].set_xlabel('Observações')

plt.tight_layout()
plt.show()

print("\n💡 Interpretação:")
print("- Colunas com muito missing (>20%) podem precisar de tratamento especial")
print("- Observe se há padrões (missing acontece junto em certas linhas?)")

## 🔍 Classificação de Missing: MCAR, MAR ou MNAR?

### Tipos de Missing Data:
- **MCAR (Missing Completely At Random):** Valores faltam aleatoriamente, sem relação com outros dados
- **MAR (Missing At Random):** Missing relacionado a outras variáveis observadas
- **MNAR (Missing Not At Random):** Missing relacionado ao próprio valor não observado

Vamos investigar cada coluna com missing:

In [ ]:
# === INVESTIGAR MISSING EM PRICE ===
print("=" * 60)
print("ANÁLISE: PRICE MISSING")
print("=" * 60)

# Comparar grupos com/sem price
with_price = df[df['price'].notna()]
without_price = df[df['price'].isna()]

print(f"\nRegistros COM price: {len(with_price):,}")
print(f"Registros SEM price: {len(without_price):,}")

# Comparar características
print("\n📊 Comparação de características:")
print("\nÁrea média:")
#print(f"  COM price: {with_price['area'].mean():.2f} m²")
print(f"  COM price: {pd.to_numeric(with_price['area'], errors='coerce').mean():.2f} m²")
#print(f"  SEM price: {without_price['area'].mean():.2f} m²")
print(f"  SEM price: {pd.to_numeric(without_price['area'], errors='coerce').mean():.2f} m²")


print("\nIdade média:")
# print(f"  COM price: {with_price['age'].mean():.2f} anos")
# print(f"  SEM price: {without_price['age'].mean():.2f} anos")
print(f"  COM price: {pd.to_numeric(with_price['age'], errors='coerce').mean():.2f} anos")
print(f"  SEM price: {pd.to_numeric(without_price['age'], errors='coerce').mean():.2f} anos")
# Visualizar distribuições
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(with_price['area'].dropna(), bins=30, alpha=0.7, label='COM price', color='green')
axes[0].hist(without_price['area'].dropna(), bins=30, alpha=0.7, label='SEM price', color='red')
axes[0].set_xlabel('Área (m²)')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição de Área')
axes[0].legend()

axes[1].hist(with_price['age'].dropna(), bins=30, alpha=0.7, label='COM price', color='green')
axes[1].hist(without_price['age'].dropna(), bins=30, alpha=0.7, label='SEM price', color='red')
axes[1].set_xlabel('Idade (anos)')
axes[1].set_ylabel('Frequência')
axes[1].set_title('Distribuição de Idade')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n💡 Interpretação:")
print("Se as distribuições forem DIFERENTES → pode ser MAR ou MNAR")
print("Se as distribuições forem SIMILARES → provavelmente MCAR")

In [ ]:
# === DOCUMENTAR CLASSIFICAÇÃO DE MISSING ===
missing_classification = {
    'price': {
        'type': 'MNAR',
        'reason': 'Imóveis caros em negociação podem não ter preço publicado',
        'evidence': 'Imóveis sem preço tendem a ter área maior',
        'strategy': 'Imputar por grupo (neighborhood) + criar flag'
    },
    'area': {
        'type': 'MCAR',
        'reason': 'Medições não realizadas ainda de forma aleatória',
        'evidence': 'Distribuição similar entre grupos com/sem area',
        'strategy': 'Imputar com mediana por neighborhood'
    },
    'age': {
        'type': 'MNAR',
        'reason': 'Imóveis muito antigos sem registro de construção',
        'evidence': 'Lógica do negócio - dados históricos perdidos',
        'strategy': 'Valor alto fixo (100 anos) + criar flag'
    },
    'bedrooms': {
        'type': 'MCAR',
        'reason': 'Informação não coletada aleatoriamente',
        'evidence': 'Pequeno percentual, sem padrão aparente',
        'strategy': 'Imputar com mediana global'
    },
    'bathrooms': {
        'type': 'MCAR',
        'reason': 'Informação não coletada aleatoriamente',
        'evidence': 'Pequeno percentual, sem padrão aparente',
        'strategy': 'Imputar com mediana global'
    }
}

print("=" * 60)
print("CLASSIFICAÇÃO E ESTRATÉGIA PARA CADA VARIÁVEL")
print("=" * 60)
for col, info in missing_classification.items():
    if col in missing_report['Column'].values:
        print(f"\n📌 {col.upper()}")
        print(f"   Tipo: {info['type']}")
        print(f"   Razão: {info['reason']}")
        print(f"   Evidência: {info['evidence']}")
        print(f"   Estratégia: {info['strategy']}")

## 🔍 Outros Problemas Além de Missing

In [ ]:
# === VERIFICAR TIPOS DE DADOS ===
print("=" * 60)
print("TIPOS DE DADOS ATUAIS")
print("=" * 60)
print(df.dtypes)

print("\n⚠ PROBLEMAS IDENTIFICADOS:")
print("\n1. 'age' e 'area' estão como 'object' (deveriam ser numéricos)")
print("   → Contêm valores inválidos: 'Unknown', '???'")
print("\n2. Algumas colunas categóricas podem ter inconsistências")
print("   → Espaços extras, maiúsculas/minúsculas inconsistentes")

In [ ]:
# === IDENTIFICAR VALORES INVÁLIDOS ===
print("=" * 60)
print("VALORES ÚNICOS EM AGE (coluna object)")
print("=" * 60)
age_unique = df['age'].value_counts().head(20)
print(age_unique)

print("\n⚠ Valores inválidos encontrados em 'age': Unknown, ???")

print("\n" + "=" * 60)
print("VALORES ÚNICOS EM AREA (coluna object)")
print("=" * 60)
area_unique = df['area'].value_counts().head(20)
print(area_unique)

print("\n⚠ Valores inválidos encontrados em 'area': Unknown, ???")

In [ ]:
# === VERIFICAR INCONSISTÊNCIAS EM STRINGS ===
print("=" * 60)
print("VALORES ÚNICOS EM 'condition'")
print("=" * 60)
print(df['condition'].value_counts())

print("\n⚠ PROBLEMAS ENCONTRADOS:")
print("1. Espaços extras: 'fair ', ' fair'")
print("2. Capitalização inconsistente: 'good', 'Good', 'GOOD'")
print("3. Variações: 'bom' (português) vs 'good' (inglês)")

print("\n" + "=" * 60)
print("VALORES ÚNICOS EM 'neighborhood'")
print("=" * 60)
print(f"Total de bairros únicos: {df['neighborhood'].nunique()}")
print("\nPrimeiros 15:")
print(df['neighborhood'].value_counts().head(15))

In [ ]:
# === VERIFICAR VALORES SUSPEITOS/OUTLIERS ===
print("=" * 60)
print("VALORES SUSPEITOS EM PRICE")
print("=" * 60)
print(f"Valor mínimo: R$ {df['price'].min():,.2f}")
print(f"Valor máximo: R$ {df['price'].max():,.2f}")
print(f"\nPreços acima de 10 milhões: {(df['price'] > 10000000).sum()}")
print(f"Preços abaixo de 10 mil: {(df['price'] < 10000).sum()}")

# Mostrar os outliers extremos
extreme_prices = df[df['price'] > 10000000][['price', 'area', 'bedrooms', 'neighborhood']]
if len(extreme_prices) > 0:
    print("\n⚠ Outliers extremos:")
    display(extreme_prices)

In [ ]:
# === VERIFICAR DUPLICATAS ===
print("=" * 60)
print("ANÁLISE DE DUPLICATAS")
print("=" * 60)
n_duplicates = df.duplicated().sum()
print(f"\nRegistros duplicados (todas as colunas): {n_duplicates}")

# Verificar duplicatas ignorando sale_date
cols_without_date = [col for col in df.columns if col != 'sale_date']
n_duplicates_no_date = df.duplicated(subset=cols_without_date).sum()
print(f"Registros duplicados (ignorando sale_date): {n_duplicates_no_date}")

if n_duplicates > 0:
    print("\n⚠ Encontradas duplicatas! Devem ser removidas.")

---
# FASE 3: Planejamento e Execução (Plan & Execute)
**Objetivo:** Aplicar estratégias de limpeza sistematicamente

In [ ]:
# === CRIAR CÓPIA PARA PRESERVAR ORIGINAL ===
df_original = df.copy()
print("✓ Cópia do dataset original salva em 'df_original'")
print("  Sempre preserve os dados originais!")

## PASSO 1: Limpeza de Strings

## 📝 Exercício 2: Limpar Strings Categóricas

Complete o código para limpar as colunas categóricas:

In [ ]:
# === EXERCÍCIO 2: LIMPEZA DE STRINGS ===
print("=" * 60)
print("PASSO 1: LIMPEZA DE STRINGS")
print("=" * 60)

# Colunas categóricas para limpar
categorical_cols = ['neighborhood', 'condition', 'has_garden']

print("\nANTES da limpeza:")
for col in categorical_cols:
    print(f"\n{col}: {df[col].nunique()} valores únicos")
    print(f"Exemplos: {df[col].unique()[:5]}")

# === SEU CÓDIGO AQUI ===
# Para cada coluna categórica:
# 1. Remova espaços extras com .str.strip()
# 2. Converta para minúsculo com .str.lower()

for col in categorical_cols:
    df[col] = df[col].str.strip().str.lower().str.replace(' ', '', regex=False)
    # df[col] = df[col].str.replace(r'\s+', '', regex=True).str.lower()

print("\n" + "-" * 60)
print("DEPOIS da limpeza:")
for col in categorical_cols:
    print(f"\n{col}: {df[col].nunique()} valores únicos")
    print(f"Exemplos: {df[col].unique()[:5]}")

print("\n✓ Strings limpas: espaços removidos e texto padronizado")

In [ ]:
# === TESTAR EXERCÍCIO 2 ===
grader.check("p3")

## PASSO 2: Conversão de Tipos e Tratamento de Inválidos

## 📝 Exercício 3: Converter Tipos e Tratar Valores Inválidos

Use `pd.to_numeric()` para converter as colunas 'age' e 'area' para numérico, transformando valores inválidos em NaN:

In [ ]:
# === EXERCÍCIO 3: CONVERSÃO DE TIPOS ===
print("=" * 60)
print("PASSO 2: CONVERSÃO DE TIPOS")
print("=" * 60)

print("\nANTES da conversão:")
print(f"age dtype: {df['age'].dtype}")
print(f"area dtype: {df['area'].dtype}")
print(f"\nExemplos de valores inválidos em age: {df['age'][df['age'].isin(['Unknown', '???'])].head()}")

# === SEU CÓDIGO AQUI ===
# Use pd.to_numeric() com errors='coerce' para:
# 1. Converter 'age' para numérico
# 2. Converter 'area' para numérico
# Valores inválidos (Unknown, ???) se tornarão NaN

df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['area'] = pd.to_numeric(df['area'], errors='coerce')

print("\n" + "-" * 60)
print("DEPOIS da conversão:")
print(f"age dtype: {df['age'].dtype}")
print(f"area dtype: {df['area'].dtype}")
print(f"\nNovos NaNs criados em age: {df['age'].isnull().sum()}")
print(f"Novos NaNs criados em area: {df['area'].isnull().sum()}")

print("\n✓ Tipos convertidos: valores inválidos transformados em NaN")

In [ ]:
# === TESTAR EXERCÍCIO 3 ===
grader.check("p4")

## PASSO 3: Validação de Ranges e Tratamento de Outliers

In [ ]:
# === VALIDAR RANGES E TRATAR OUTLIERS ===
print("=" * 60)
print("PASSO 3: VALIDAÇÃO DE RANGES")
print("=" * 60)

# Definir regras de negócio
validation_rules = {
    'price': (10000, 10000000),      # Entre R$ 10k e R$ 10M
    'area': (20, 1000),               # Entre 20m² e 1000m²
    'bedrooms': (0, 10),              # Entre 0 e 10 quartos
    'bathrooms': (0, 10),             # Entre 0 e 10 banheiros
    'age': (0, 150),                  # Entre 0 e 150 anos
    'garage_spaces': (0, 10)          # Entre 0 e 10 vagas
}

print("\nRegras de validação definidas:")
for col, (min_val, max_val) in validation_rules.items():
    print(f"  {col}: [{min_val}, {max_val}]")

# Aplicar validações
print("\n" + "-" * 60)
print("Aplicando validações...")
print("-" * 60)

for col, (min_val, max_val) in validation_rules.items():
    if col in df.columns:
        # Identificar valores fora do range
        invalid_mask = ~df[col].between(min_val, max_val)
        n_invalid = invalid_mask.sum()

        if n_invalid > 0:
            print(f"\n⚠ {col}: {n_invalid} valores fora do range [{min_val}, {max_val}]")
            print(f"  Exemplos: {df.loc[invalid_mask, col].dropna().head().tolist()}")

            # Substituir por NaN (serão imputados depois)
            df.loc[invalid_mask, col] = np.nan
            print(f"  ✓ Substituídos por NaN")

print("\n✓ Validação de ranges concluída")

## PASSO 4: Criar Flags de Imputação

## 📝 Exercício 4: Criar Flags de Missing

Crie flags binárias para registrar quais valores serão imputados:

In [ ]:
# === EXERCÍCIO 4: CRIAR FLAGS DE IMPUTAÇÃO ===
print("=" * 60)
print("PASSO 4: CRIAR FLAGS DE IMPUTAÇÃO")
print("=" * 60)

# Colunas que serão imputadas
cols_to_flag = ['price', 'area', 'age', 'bedrooms', 'bathrooms']

print("\nCriando flags...")

# === SEU CÓDIGO AQUI ===
# Para cada coluna em cols_to_flag:
# Crie uma nova coluna chamada '{coluna}_was_missing'
# que contenha 1 se o valor era missing, 0 caso contrário
# Use: df[col].isnull().astype(int)

for col in cols_to_flag:
    flag_name = f"{col}_was_missing"
    df[flag_name] = df[col].isnull().astype(int)
    n_missing = df[flag_name].sum()
    print(f"  ✓ {flag_name}: {n_missing} registros marcados")

print("\n✓ Flags criadas com sucesso")
print("\n💡 Por que criar flags?")
print("   → Preservar informação de que o valor estava faltando")
print("   → Modelos de ML podem usar isso como feature")
print("   → Documentação: sabemos quais valores foram imputados")

In [ ]:
# === TESTAR EXERCÍCIO 4 ===
grader.check("p2")

## PASSO 5: Imputação de Valores Faltantes

In [ ]:
# === IMPUTAÇÃO POR GRUPO (GROUPBY + TRANSFORM) ===
print("=" * 60)
print("PASSO 5A: IMPUTAÇÃO POR GRUPO")
print("=" * 60)

print("\nImputando 'price' por neighborhood (mediana):")
df['price'] = df.groupby('neighborhood')['price'].transform(
    lambda x: x.fillna(x.median())
)
print(f"  ✓ price: {df['price'].isnull().sum()} missing restantes")

print("\nImputando 'area' por neighborhood (mediana):")
df['area'] = df.groupby('neighborhood')['area'].transform(
    lambda x: x.fillna(x.median())
)
print(f"  ✓ area: {df['area'].isnull().sum()} missing restantes")

print("\n💡 Por que por grupo?")
print("   → Imóveis no mesmo bairro têm características similares")
print("   → Mais preciso que usar a média/mediana global")

In [ ]:
# === IMPUTAÇÃO GLOBAL ===
print("=" * 60)
print("PASSO 5B: IMPUTAÇÃO GLOBAL")
print("=" * 60)

# Se ainda há missing após imputação por grupo, usar global
if df['price'].isnull().sum() > 0:
    global_price_median = df['price'].median()
    df['price'].fillna(global_price_median, inplace=True)
    print(f"  ✓ price: imputados {df['price'].isnull().sum()} valores com mediana global")

if df['area'].isnull().sum() > 0:
    global_area_median = df['area'].median()
    df['area'].fillna(global_area_median, inplace=True)
    print(f"  ✓ area: imputados valores com mediana global")

# Age: valor fixo alto (MNAR - imóveis antigos)
print("\nImputando 'age' com 100 anos (imóveis muito antigos):")
df['age'].fillna(100, inplace=True)
print(f"  ✓ age: {df['age'].isnull().sum()} missing restantes")

# Bedrooms e bathrooms: mediana
print("\nImputando 'bedrooms' e 'bathrooms' com mediana:")
df['bedrooms'].fillna(df['bedrooms'].median(), inplace=True)
df['bathrooms'].fillna(df['bathrooms'].median(), inplace=True)
print(f"  ✓ bedrooms: {df['bedrooms'].isnull().sum()} missing restantes")
print(f"  ✓ bathrooms: {df['bathrooms'].isnull().sum()} missing restantes")

In [ ]:
# === FORWARD FILL E BACKWARD FILL (PARA SÉRIES TEMPORAIS) ===
print("=" * 60)
print("PASSO 5C: FORWARD/BACKWARD FILL")
print("=" * 60)

print("\n💡 Demonstração: Forward Fill (ffill) e Backward Fill (bfill)")
print("   Útil para dados ordenados temporalmente\n")

# Criar exemplo temporário para demonstração
demo_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=10, freq='D'),
    'price': [100, np.nan, np.nan, 150, np.nan, 200, np.nan, np.nan, 250, np.nan]
})

print("Dados originais:")
print(demo_data)

print("\nCom Forward Fill (propaga último valor válido):")
demo_ffill = demo_data.copy()
demo_ffill['price'] = demo_ffill['price'].ffill()
print(demo_ffill)

print("\nCom Backward Fill (propaga próximo valor válido):")
demo_bfill = demo_data.copy()
demo_bfill['price'] = demo_bfill['price'].bfill()
print(demo_bfill)

# Aplicar no nosso dataset (se necessário)
# Ordenar por sale_date antes
df_sorted = df.sort_values('sale_date')
# Exemplo: se garage_spaces tiver missing, podemos usar ffill
if df_sorted['garage_spaces'].isnull().sum() > 0:
    print("\nAplicando ffill em garage_spaces...")
    df['garage_spaces'] = df_sorted['garage_spaces'].ffill().bfill()
    print(f"  ✓ garage_spaces: {df['garage_spaces'].isnull().sum()} missing restantes")

## PASSO 6: Remover Duplicatas

In [ ]:
# === REMOVER DUPLICATAS ===
print("=" * 60)
print("PASSO 6: REMOVER DUPLICATAS")
print("=" * 60)

n_before = len(df)
print(f"\nRegistros antes: {n_before:,}")

# Remover duplicatas exatas
df = df.drop_duplicates()

n_after = len(df)
n_removed = n_before - n_after

print(f"Registros depois: {n_after:,}")
print(f"Duplicatas removidas: {n_removed}")

if n_removed > 0:
    print("\n✓ Duplicatas removidas com sucesso")
else:
    print("\n✓ Nenhuma duplicata encontrada")

## PASSO 7: Conversão Final de Tipos

In [ ]:
# === CONVERSÃO FINAL DE TIPOS ===
print("=" * 60)
print("PASSO 7: CONVERSÃO FINAL DE TIPOS")
print("=" * 60)

print("\nConvertendo para tipos apropriados...")

# Inteiros
df['bedrooms'] = df['bedrooms'].astype('int')
df['bathrooms'] = df['bathrooms'].astype('int')
df['age'] = df['age'].astype('int')
df['garage_spaces'] = df['garage_spaces'].astype('int')

# Float
df['price'] = df['price'].astype('float')
df['area'] = df['area'].astype('float')

# Categóricos (economiza memória)
df['neighborhood'] = df['neighborhood'].astype('category')
df['condition'] = df['condition'].astype('category')
df['has_garden'] = df['has_garden'].astype('category')

# Data
df['sale_date'] = pd.to_datetime(df['sale_date'])

print("\n" + "-" * 60)
print("Tipos finais:")
print("-" * 60)
print(df.dtypes)

print("\n✓ Tipos convertidos com sucesso")

---
# FASE 4: Validação Final
**Objetivo:** Verificar se a limpeza foi bem-sucedida

In [ ]:
# === VALIDAÇÃO FINAL ===
print("=" * 60)
print("RELATÓRIO FINAL DE QUALIDADE")
print("=" * 60)

print("\n📊 COMPARAÇÃO ANTES vs DEPOIS")
print("-" * 60)

# Missing values
missing_before = df_original.isnull().sum().sum()
missing_after = df.isnull().sum().sum()
print(f"\nMissing values:")
print(f"  Antes: {missing_before:,}")
print(f"  Depois: {missing_after:,}")
print(f"  Redução: {missing_before - missing_after:,} ({((missing_before - missing_after) / missing_before * 100):.1f}%)")

# Registros
rows_before = len(df_original)
rows_after = len(df)
print(f"\nRegistros:")
print(f"  Antes: {rows_before:,}")
print(f"  Depois: {rows_after:,}")
print(f"  Removidos: {rows_before - rows_after:,}")

# Categorias únicas
print(f"\nCategorias únicas em 'condition':")
print(f"  Antes: {df_original['condition'].nunique()}")
print(f"  Depois: {df['condition'].nunique()}")

# Verificações finais
print("\n" + "=" * 60)
print("CHECKLIST FINAL")
print("=" * 60)

checks = [
    ("Sem missing em price", df['price'].isnull().sum() == 0),
    ("Sem missing em area", df['area'].isnull().sum() == 0),
    ("Sem missing em age", df['age'].isnull().sum() == 0),
    ("Todos os tipos corretos", pd.api.types.is_numeric_dtype(df['price'])),
    ("Sem valores negativos em price", (df['price'] >= 0).all()),
    ("Sem valores negativos em area", (df['area'] >= 0).all()),
    ("Flags de imputação criadas", 'price_was_missing' in df.columns),
    ("Strings limpas (condition)", not df['condition'].str.contains(' ').any()),
]

all_passed = True
for check_name, check_result in checks:
    status = "✓" if check_result else "✗"
    print(f"{status} {check_name}")
    if not check_result:
        all_passed = False

if all_passed:
    print("\n" + "=" * 60)
    print("🎉 DATASET LIMPO COM SUCESSO!")
    print("=" * 60)
else:
    print("\n" + "=" * 60)
    print("⚠ Algumas verificações falharam. Revise o processo.")
    print("=" * 60)

## 📝 Exercício 5: Pipeline Completo

Agora vamos criar uma função que encapsula todo o pipeline de limpeza:

In [ ]:
# === EXERCÍCIO 5: PIPELINE COMPLETO ===

def clean_housing_pipeline(df_raw):
    """
    Pipeline completo de limpeza de dados do Housing dataset

    Parameters:
    -----------
    df_raw : DataFrame
        Dataset bruto a ser limpo

    Returns:
    --------
    DataFrame limpo e pronto para análise
    """
    print("🚀 Iniciando pipeline de limpeza...\n")

    # Criar cópia
    df_clean = df_raw.copy()

    # === SEU CÓDIGO AQUI ===
    # Implemente os 7 passos do pipeline:

    # PASSO 1: Limpeza de strings
    print("[1/7] Limpando strings...")
    categorical_cols = ['neighborhood', 'condition', 'has_garden']
    for col in categorical_cols:
        df_clean[col] = df_clean[col].str.strip().str.lower()

    # PASSO 2: Conversão de tipos
    print("[2/7] Convertendo tipos...")
    df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
    df_clean['area'] = pd.to_numeric(df_clean['area'], errors='coerce')

    # PASSO 3: Validação de ranges
    print("[3/7] Validando ranges...")
    validation_rules = {
        'price': (10000, 10000000),
        'area': (20, 1000),
        'bedrooms': (0, 10),
        'bathrooms': (0, 10),
        'age': (0, 150),
        'garage_spaces': (0, 10)
    }
    for col, (min_val, max_val) in validation_rules.items():
        if col in df_clean.columns:
            invalid_mask = ~df_clean[col].between(min_val, max_val)
            df_clean.loc[invalid_mask, col] = np.nan

    # PASSO 4: Criar flags
    print("[4/7] Criando flags de imputação...")
    for col in ['price', 'area', 'age', 'bedrooms', 'bathrooms']:
        df_clean[f"{col}_was_missing"] = df_clean[col].isnull().astype(int)

    # PASSO 5: Imputação
    print("[5/7] Imputando valores...")
    # Por grupo
    df_clean['price'] = df_clean.groupby('neighborhood')['price'].transform(
        lambda x: x.fillna(x.median())
    )
    df_clean['area'] = df_clean.groupby('neighborhood')['area'].transform(
        lambda x: x.fillna(x.median())
    )
    # Global
    df_clean['price'].fillna(df_clean['price'].median(), inplace=True)
    df_clean['area'].fillna(df_clean['area'].median(), inplace=True)
    df_clean['age'].fillna(100, inplace=True)
    df_clean['bedrooms'].fillna(df_clean['bedrooms'].median(), inplace=True)
    df_clean['bathrooms'].fillna(df_clean['bathrooms'].median(), inplace=True)

    # PASSO 6: Remover duplicatas
    print("[6/7] Removendo duplicatas...")
    df_clean = df_clean.drop_duplicates()

    # PASSO 7: Conversão final de tipos
    print("[7/7] Convertendo tipos finais...")
    df_clean['bedrooms'] = df_clean['bedrooms'].astype('int')
    df_clean['bathrooms'] = df_clean['bathrooms'].astype('int')
    df_clean['age'] = df_clean['age'].astype('int')
    df_clean['garage_spaces'] = df_clean['garage_spaces'].astype('int')
    df_clean['neighborhood'] = df_clean['neighborhood'].astype('category')
    df_clean['condition'] = df_clean['condition'].astype('category')
    df_clean['has_garden'] = df_clean['has_garden'].astype('category')
    df_clean['sale_date'] = pd.to_datetime(df_clean['sale_date'])

    print("\n✓ Pipeline concluído com sucesso!\n")
    return df_clean

# Testar o pipeline
print("=" * 60)
print("TESTANDO PIPELINE COMPLETO")
print("=" * 60)
df_clean = clean_housing_pipeline(df_original)

print("\nResultado:")
print(f"  Registros: {len(df_clean):,}")
print(f"  Missing total: {df_clean.isnull().sum().sum()}")
print(f"  Colunas: {len(df_clean.columns)}")

In [ ]:
# === TESTAR EXERCÍCIO 5 ===
grader.check("p5")

## 📊 Visualização: Antes vs Depois

In [ ]:
# === COMPARAÇÃO VISUAL ANTES VS DEPOIS ===
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribuição de preços
axes[0, 0].hist(df_original['price'].dropna(), bins=50, alpha=0.7, label='Antes', color='red')
axes[0, 0].hist(df_clean['price'], bins=50, alpha=0.7, label='Depois', color='green')
axes[0, 0].set_xlabel('Preço (R$)')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].set_title('Distribuição de Preços')
axes[0, 0].legend()

# 2. Distribuição de área
axes[0, 1].hist(pd.to_numeric(df_original['area'], errors='coerce').dropna(), bins=50, alpha=0.7, label='Antes', color='red')
axes[0, 1].hist(df_clean['area'], bins=50, alpha=0.7, label='Depois', color='green')
axes[0, 1].set_xlabel('Área (m²)')
axes[0, 1].set_ylabel('Frequência')
axes[0, 1].set_title('Distribuição de Área')
axes[0, 1].legend()

# 3. Missing por coluna
missing_before = df_original.isnull().sum().sort_values(ascending=False).head(10)
missing_after = df_clean.isnull().sum().sort_values(ascending=False).head(10)
x = np.arange(len(missing_before))
width = 0.35
axes[1, 0].bar(x - width/2, missing_before.values, width, label='Antes', color='red', alpha=0.7)
axes[1, 0].bar(x + width/2, missing_after.values, width, label='Depois', color='green', alpha=0.7)
axes[1, 0].set_xlabel('Colunas')
axes[1, 0].set_ylabel('Quantidade de Missing')
axes[1, 0].set_title('Missing Values por Coluna')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(missing_before.index, rotation=45, ha='right')
axes[1, 0].legend()

# 4. Categorias em condition
condition_before = df_original['condition'].value_counts()
condition_after = df_clean['condition'].value_counts()
axes[1, 1].barh(range(len(condition_before)), condition_before.values, alpha=0.7, label='Antes', color='red')
axes[1, 1].barh(range(len(condition_after)), condition_after.values, alpha=0.7, label='Depois', color='green')
axes[1, 1].set_xlabel('Frequência')
axes[1, 1].set_ylabel('Condição')
axes[1, 1].set_title('Categorias em Condition')
axes[1, 1].set_yticks(range(len(condition_after)))
axes[1, 1].set_yticklabels(condition_after.index)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

---
# 📚 Resumo do Aprendizado

## O que você aprendeu:

### 1. **Identificação de Problemas**
- Como detectar valores faltantes com `isnull()`, `sum()`
- Criar relatórios de missing com percentuais
- Classificar missing: MCAR, MAR, MNAR
- Identificar valores inválidos e inconsistências

### 2. **Estratégias de Imputação**
- **Por grupo:** `groupby().transform()` - mais preciso
- **Global:** `fillna()` com média/mediana/moda
- **Temporal:** `ffill()` e `bfill()` para séries temporais
- **Valor fixo:** Quando faz sentido no contexto do negócio

### 3. **Limpeza de Strings**
- `str.strip()` - remover espaços
- `str.lower()` / `str.upper()` - padronizar capitalização
- Redução de categorias duplicadas

### 4. **Validação de Dados**
- Definir regras de negócio para ranges válidos
- `pd.to_numeric()` com `errors='coerce'` para conversão segura
- Tratar outliers e valores impossíveis

### 5. **Boas Práticas**
- ✓ Sempre preservar dados originais
- ✓ Criar flags de imputação
- ✓ Documentar todas as decisões
- ✓ Validar resultados sistematicamente
- ✓ Construir pipelines reproduzíveis

## Próximos Passos:
- Análise exploratória dos dados limpos
- Feature engineering
- Modelagem preditiva

---

## 🎯 Conceitos-Chave para Lembrar:

1. **80% do tempo em Data Science é limpeza de dados** - não pule esta etapa!
2. **Entenda o contexto do negócio** antes de tomar decisões de limpeza
3. **Missing data não é sempre aleatório** - classificar como MCAR/MAR/MNAR é crucial
4. **Sempre documente** suas decisões e transformações
5. **Crie pipelines reproduzíveis** para facilitar manutenção e atualização

In [ ]:
# === SALVAR DATASET LIMPO ===
output_filename = 'housing_prices_clean.csv'
df_clean.to_csv(output_filename, index=False)
print(f"✓ Dataset limpo salvo como '{output_filename}'")

# Estatísticas finais
print("\n" + "=" * 60)
print("ESTATÍSTICAS FINAIS")
print("=" * 60)
print(f"Registros: {len(df_clean):,}")
print(f"Colunas: {len(df_clean.columns)}")
print(f"Missing total: {df_clean.isnull().sum().sum()}")
print(f"Memória: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n🎉 Parabéns! Você completou o pipeline de limpeza de dados!")

In [ ]:
# === VERIFICAÇÃO FINAL DE TODOS OS TESTES ===
print("=" * 60)
print("EXECUTANDO TODOS OS TESTES")
print("=" * 60)
grader.check_all()